# EDA - Marine_Microplastics 🌊

## Importación de librerías y datos 📁

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
df_mp = pd.read_csv("../../Archivos/Marine_Microplastics.csv")
df_mp.head()

,OBJECTID,Oceans,Regions,SubRegions,Sampling Method,Measurement,Unit,Density Range,Density Class,Short Reference,Long Reference,DOI,Organization,Keywords,Accession Number,Accession Link,Latitude,Longitude,Date,GlobalID,x,y
0,9676,Atlantic Ocean,NaN,NaN,Grab sample,0.018,pieces/m3,0.005-1,Medium,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,-31.696000,-48.560000,8/11/2015 12:00:00 AM,a77121b2-e113-444e-82d9-7af11d62fdd2,-48.560000,-31.696000
1,6427,Pacific Ocean,NaN,NaN,Neuston net,0.000,pieces/m3,0-0.0005,Very Low,Law et al.2014,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",https://doi.org/10.1021/es4053076,Sea Education Association,SEA,211008,https://www.ncei.noaa.gov/access/metadata/land...,6.350000,-121.850000,12/18/2002 12:00:00 AM,be27c450-02ca-4261-8d89-cae21108e6cc,-121.850000,6.350000
2,10672,Pacific Ocean,NaN,NaN,Manta net,0.013,pieces/m3,0.005-1,Medium,Goldstein et al.2013,"Goldstein, M.C., A.J. Titmus, M. Ford. 2013. S...",https://doi.org/10.1371/journal.pone.0080020,Scripps Institution of Oceanography-University...,Great Pacific Garbage Patch/SEAPLEX,253448,https://www.ncei.noaa.gov/access/metadata/land...,0.500000,-95.350000,10/17/2006 12:00:00 AM,23effcdd-35b7-4e1e-adb4-390693a287d3,-95.350000,0.500000
3,13921,Atlantic Ocean,NaN,NaN,Aluminum bucket,1368.000,pieces/m3,>=10,Very High,Queiroz et al.2022,"Queiroz, A.F.dos S., A.S. da Conceição, D. Che...",https://doi.org/10.1016/j.scitotenv.2022.156259,"Federal University of Pará, Brazil",Amazon Continental Shelf,276482,https://www.ncei.noaa.gov/access/metadata/land...,0.631825,-45.398158,10/17/2018 12:00:00 AM,16d77822-0533-4116-97b9-0bdb592f3d6e,-45.398158,0.631825
4,9344,Pacific Ocean,NaN,NaN,Grab sample,0.001,pieces/m3,0.0005-0.005,Low,Barrows et al.2018,"Barrows, A.P.W., S.E. Cathey, C.W. Petersen. 2...",https://doi.org/10.1016/j.envpol.2018.02.062,Adventure Scientist,Adventure Scientist/Citizen Science,211009,https://www.ncei.noaa.gov/access/metadata/land...,16.623000,-99.697800,1/3/2015 12:00:00 AM,b9e435e3-9e86-4143-8b51-877e5dcdc7a6,-99.697800,16.623000


## EDA 🧼🫧

### Primera exploración 🔎

In [3]:
# Forma del conjunto de datos, en (filas, columnas):
df_mp.shape

(20425, 22)

In [4]:
# Función para conocer nulos y duplicados en un informe. Next step--> ETL

def nulos_duplicados(df_mp):
    # Cálculo del porcentaje de nulos
    porcentaje_nulos = df_mp.isna().sum() / df_mp.shape[0] * 100
    
    # Verificación de duplicados
    duplicados = df_mp.duplicated().sum()
    if duplicados == 0:
        mensaje_duplicados = "No hay duplicados"
    else:
        mensaje_duplicados = f"Hay {duplicados} duplicados"
    
    # Creación de un reporte bonito y visual
    reporte = f"""
    ===================== Informe de Datos =====================
    
    Porcentaje de Nulos por Columna:
    ------------------------------------------------------------
    {porcentaje_nulos.to_string()}
    
    ------------------------------------------------------------
    Duplicados:
    ------------------------------------------------------------
    {mensaje_duplicados}
    
    ============================================================
    """
    
    # Imprimir directamente el reporte
    print(reporte)

# Ejemplo de uso
# df_mp = pd.DataFrame(...)

# Llamar directamente a la función
nulos_duplicados(df_mp)



    ===================== Informe de Datos =====================
    
    Porcentaje de Nulos por Columna:
    ------------------------------------------------------------
    OBJECTID             0.000000
Oceans               1.326805
Regions             56.499388
SubRegions          93.600979
Sampling Method      0.000000
Measurement         28.455324
Unit                 0.000000
Density Range        0.000000
Density Class        0.000000
Short Reference      0.000000
Long Reference       0.000000
DOI                  0.000000
Organization         0.000000
Keywords             0.088127
Accession Number     0.000000
Accession Link       0.000000
Latitude             0.000000
Longitude            0.000000
Date                 0.000000
GlobalID             0.000000
x                    0.000000
y                    0.000000
    
    ------------------------------------------------------------
    Duplicados:
    ------------------------------------------------------------
    No hay d

### Gestión de duplicados y nulos ✏️

In [5]:
# Gestión de duplicados: No procede

In [6]:
# Gestión de nulos:

In [7]:
#Ocean
#Relacionar oceanos con Longitud y Latitud (COMPROBACIONES)
df_mp[df_mp['Oceans'].notnull()].groupby('Oceans')[['Latitude', 'Longitude']].mean().sort_values('Latitude')

,Latitude,Longitude
Oceans,,
Indian Ocean,-26.203959,28.731356
Pacific Ocean,23.412917,-137.379422
Atlantic Ocean,28.981572,-65.054741
Arctic Ocean,77.023468,-40.293646


In [8]:
# Primero, voy a crear una columna nueva categórica, para crear un map en los oceanos, y poder imputar con latitud

df_mp["Oceans"].unique()
dicc ={'Atlantic Ocean':1,'Pacific Ocean':2,'Arctic Ocean':3,'Indian Ocean':4, "nan":None}
df_mp['Oceans_cat']=df_mp['Oceans'].map(dicc)

In [9]:
# Imputación:

imputer_knn = KNNImputer(n_neighbors=3)
imputado = imputer_knn.fit_transform(df_mp[["Oceans_cat", "Latitude"]])
df_mp[["Ocean_cat_knn", "Latitude_knn"]] = imputado

#Borramos las columnas que ya no nos interesan
df_mp = df_mp.drop(columns=["Oceans_cat", "Latitude_knn"])
#Revertimos el proceso, para asignar a cada numero imputado, el Oceano correspondiente
dicc2 ={1:'Atlantic Ocean',2:"Pacific Ocean",3:'Arctic Ocean',4:'Indian Ocean'}
df_mp['Oceans'] = df_mp['Oceans'].fillna(df_mp['Ocean_cat_knn'].round().astype(int).map(dicc2))
# Borramos columnas no relevantes
df_mp=df_mp.drop(columns=["Ocean_cat_knn"])

""" la columna "Oceans" fue restaurada, y 
los valores nulos fueron reemplazados por los océanos 
correspondientes, obteniendo una columna sin valores 
nulos. """


' la columna "Oceans" fue restaurada, y \nlos valores nulos fueron reemplazados por los océanos \ncorrespondientes, obteniendo una columna sin valores \nnulos. '

In [10]:
# Comprobaciones varias:

df_mp["Sampling Method"].unique()

array(['Grab sample', 'Neuston net', 'Manta net', 'Aluminum bucket',
       'Megacorer', 'Intake seawater pump', 'Hand picking',
       'PVC cylinder', 'Van Dorn sampler', 'Remotely operated vehicle',
       'Metal spoon', 'AVANI net', 'Plankton net', 'Day grab',
       'Stainless steel spoon', 'Shipek grab sampler',
       'Petite Ponar benthic grab', 'Van Veen grab sampler',
       'CTD rosette sampler'], dtype=object)

In [11]:
# Ver si hay relación entre Sampling Method y Unit.

In [12]:
# Comprobaciones varias:
df_mp["Unit"].unique()

array(['pieces/m3', 'pieces kg-1 d.w.', 'pieces/10 mins'], dtype=object)

In [13]:
# Comprobaciones varias:
# saber a que método corresponden las medidas
# Tabla de contingencia entre Métodos y Unidades de Medida
contingencia = pd.crosstab(df_mp['Sampling Method'], df_mp['Unit'], margins=True)
display(contingencia)

Unit,pieces kg-1 d.w.,pieces/10 mins,pieces/m3,All
Sampling Method,,,,
AVANI net,0,0,18,18
Aluminum bucket,0,0,57,57
CTD rosette sampler,0,0,36,36
Day grab,17,0,0,17
Grab sample,0,0,1181,1181
Hand picking,0,5812,18,5830
Intake seawater pump,0,0,181,181
Manta net,0,0,2342,2342
Megacorer,90,0,0,90


In [14]:
# Se puede observar, que excepto "hand picking" las demás tienen una unidad definida.
# Entonces, por la unidad, podemos sacar el método que se utilizó, obviando "hand picking"

#Como tenemos nulos en measurement, no en unit.

In [15]:
# Comprobaciones varias:

df_mp.groupby("Sampling Method")["Measurement"].apply(lambda x: x.isnull().sum())
# Podemos ver que todos los nulos son de Hand picking, "recogido a mano"--> No relevante --> No imputación

Sampling Method
AVANI net                       0
Aluminum bucket                 0
CTD rosette sampler             0
Day grab                        0
Grab sample                     0
Hand picking                 5812
Intake seawater pump            0
Manta net                       0
Megacorer                       0
Metal spoon                     0
Neuston net                     0
PVC cylinder                    0
Petite Ponar benthic grab       0
Plankton net                    0
Remotely operated vehicle       0
Shipek grab sampler             0
Stainless steel spoon           0
Van Dorn sampler                0
Van Veen grab sampler           0
Name: Measurement, dtype: int64

### Transformaciones 💻

In [16]:
# Transformaciones:

In [17]:
# Separar la columna 'Date' en fecha y hora por el espacio
# En caso de que alguna fila no tenga espacio, la función llenará con NaN
df_mp[['Date_Only', 'Time_Only']] = df_mp['Date'].str.split(' ', expand=True, n=1)

# Verificar el resultado
print(df_mp[['Date_Only', 'Time_Only']].head())


    Date_Only    Time_Only
0   8/11/2015  12:00:00 AM
1  12/18/2002  12:00:00 AM
2  10/17/2006  12:00:00 AM
3  10/17/2018  12:00:00 AM
4    1/3/2015  12:00:00 AM


In [18]:
df_mp["Time_Only"].unique()
#Todas las horas son las 12 AM, podemos prescindir de ese dato:

array(['12:00:00 AM'], dtype=object)

In [19]:
df_mp['Date'] = pd.to_datetime(df_mp['Date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [20]:
# Extraer el año
df_mp['Year'] = df_mp['Date'].dt.year

# Extraer el mes
df_mp['Month'] = df_mp['Date'].dt.month

# Extraer la hora
df_mp['Hour'] = df_mp['Date'].dt.hour


In [21]:
# Time_Only y Hour, las podemos borrar

# Siempre 12:00 AM

df_mp = df_mp.drop(columns=["Time_Only", "Hour","x","y",'SubRegions','Date_Only'])

In [22]:
# Ajuste en los valores de Regions:

df_mp['Regions'].nunique() # Número de valores únicos

47

In [23]:
# Limpiar espacios en blanco adicionales y normalizar el texto
df_mp['Regions'] = df_mp['Regions'].str.strip()  # Eliminar espacios antes y después
df_mp['Regions'] = df_mp['Regions'].str.lower()  # Convertir a minúsculas (opcional, solo si es necesario)

# Verificar los valores únicos después de la limpieza
print(df_mp['Regions'].unique())

#Porque Rio de la plata aparecía repetido

[nan 'caribbean sea' 'mediterranean sea' 'north sea'
 'inner seas off the west coast of scotland' 'new york bight'
 'gulf of mexico' 'gulf of california' 'celtic sea'
 'coastal waters of florida'
 'coastal waters of southeast alaska and british columbia'
 'gulf of alaska' 'stellwagen bank national marine sanctuary'
 "irish sea and st. george's channel" 'norwegian sea' 'kattegat'
 'english channel' 'gulf of st. lawrence' 'baltic sea'
 'papahanaumokuakea marine national monument' 'bay of biscay'
 'northwestern passages' 'monterey bay national marine sanctuary'
 'skagerrak strait'
 'the coastal waters of southeast alaska and british columbia'
 'beaufort sea' 'davis strait' 'greenland sea' 'bay of fundy'
 'gulf of bothnia' 'baffin bay' 'black sea'
 'florida keys national marine sanctuary' 'labrador sea' 'hudson strait'
 'mozambique channel' 'red sea' 'olympic coast national marine sanctuary'
 'rio de la plata' 'channel islands national marine sanctuary'
 'chukchi sea' 'bering sea' 'barents

In [24]:
df_mp['Regions'].nunique() # Obtenemos un número menos que antes

46

In [25]:
# Sacar las regiones con la latitud y la longitud, para imputar las regiones que faltan. NO CORRECTA LA IMPUTACION

# !pip install geopy

"""Geopy es una librería de Python que proporciona herramientas para interactuar con APIs
de geolocalización, como las de Google Maps, Nominatim de OpenStreetMap, y otros servicios
de geocodificación y geolocalización inversa. Permite convertir coordenadas geográficas
en direcciones (geocodificación inversa) y viceversa (geocodificación directa).
También ofrece soporte para el cálculo de distancias y la geolocalización en diferentes coordenadas."""


'Geopy es una librería de Python que proporciona herramientas para interactuar con APIs\nde geolocalización, como las de Google Maps, Nominatim de OpenStreetMap, y otros servicios\nde geocodificación y geolocalización inversa. Permite convertir coordenadas geográficas\nen direcciones (geocodificación inversa) y viceversa (geocodificación directa).\nTambién ofrece soporte para el cálculo de distancias y la geolocalización en diferentes coordenadas.'

In [26]:
# Agrupar por 'Oceans' y obtener las regiones únicas para cada océano
oceans_regions = df_mp.groupby('Oceans')['Regions'].unique()

# Mostrar el resultado
print(oceans_regions)


Oceans
Arctic Ocean      [nan, norwegian sea, northwestern passages, be...
Atlantic Ocean    [nan, caribbean sea, mediterranean sea, north ...
Indian Ocean                     [nan, mozambique channel, red sea]
Pacific Ocean     [nan, gulf of california, coastal waters of so...
Name: Regions, dtype: object


In [27]:
# Agrupar por 'Oceans' y contar las ocurrencias de cada 'Regions'
oceans_regions_count = df_mp.groupby('Oceans')['Regions'].value_counts()

# Encontrar la región más frecuente en cada océano
most_frequent_region = oceans_regions_count.groupby('Oceans').idxmax()

# Contar la frecuencia de la región más frecuente
most_frequent_count = oceans_regions_count.groupby('Oceans').max()

# Calcular el total de registros por océano
total_by_ocean = df_mp.groupby('Oceans').size()

# Calcular el porcentaje de la región más frecuente dentro de cada océano
percentage_most_frequent = (most_frequent_count / total_by_ocean) * 100

# Crear un DataFrame con la región más frecuente y su porcentaje
result = pd.DataFrame({
    'Most Frequent Region': most_frequent_region,
    'Frequency': most_frequent_count,
    'Percentage': percentage_most_frequent
})

# Mostrar el resultado
print(result)


                               Most Frequent Region  Frequency  Percentage
Oceans                                                                    
Arctic Ocean          (Arctic Ocean, greenland sea)         35   19.662921
Atlantic Ocean     (Atlantic Ocean, gulf of mexico)       4817   31.083436
Indian Ocean     (Indian Ocean, mozambique channel)          3   15.000000
Pacific Ocean   (Pacific Ocean, gulf of california)        116    2.452431


In [28]:
# No tenemos grandes porcentajes para imputar por la moda. De momento, dejamos así los datos, usaremos geopy en un futuro si es necesario

### Resultados ✅

In [30]:
# Yo creo que de momento, con este conjunto de datos limpio, podemos trabajar, para las cuestiones que se nos da:

In [31]:
# Guardar csv filtrado de Sugarcane

# df_mp.to_csv('Clean_Microplastics.csv', index=False)

In [32]:
df_mp.shape

(20425, 21)

In [33]:
nulos_duplicados(df_mp)


    ===================== Informe de Datos =====================
    
    Porcentaje de Nulos por Columna:
    ------------------------------------------------------------
    OBJECTID             0.000000
Oceans               0.000000
Regions             56.499388
Sampling Method      0.000000
Measurement         28.455324
Unit                 0.000000
Density Range        0.000000
Density Class        0.000000
Short Reference      0.000000
Long Reference       0.000000
DOI                  0.000000
Organization         0.000000
Keywords             0.088127
Accession Number     0.000000
Accession Link       0.000000
Latitude             0.000000
Longitude            0.000000
Date                 0.000000
GlobalID             0.000000
Year                 0.000000
Month                0.000000
    
    ------------------------------------------------------------
    Duplicados:
    ------------------------------------------------------------
    No hay duplicados
    
    
